# Working with lazy signals in HyperSpy

Requires **HyperSpy 1.4.2 or above**

This tutorial introduces to the processing of large dataset - which can not fit into memory - using HyperSpy. It introduce the concept of out-of-core computation algorithms (also refer as lazy processing) and the main difference between lazy and non-lazy processing as well as technicalities you need to be aware of to optimise performance.
The corresponding section of the HyperSpy documentation is [the big data section](https://hyperspy.readthedocs.io/en/stable/user_guide/big_data.html#limitations).

### Credits and changes

* 12/04/2021 Magnus Nord. Change to using 4D-STEM dataset, instead of the EELS map.
* 29/07/2019 Eric Prestat. Add more details and introduction for the M&M Sunday short course.
* 15/03/2019 Francisco de la Peña. Create tutorial for the HyperSpy workshop at ePSIC.

## Table of contents
1. [Introduction to lazy processing](#1.-Introduction-to-lazy-processing)
2. [Loading data lazily](#2.-Loading-data-lazily)
3. [Plotting lazily](#3.-Plotting-lazily)
4. [Rebinning](#4.-Rebinning)
5. [ROI in navigation dimension](#5.-ROI-in-navigation-dimension)
6. [Summary](#6.-Summary)

## 1. Introduction to lazy processing

Lazy processing refers to the use of [out-of-core computation algorithms](https://en.wikipedia.org/wiki/External_memory_algorithm) to process very large data, which are usually too large to fit into the computer's memory at one time. The main idea is to chunk the data in pieces, small enough, that can be processed in memory.

HyperSpy internally uses the [dask library](https://docs.dask.org/en/latest/index.html), which implements the numpy interface to larger-than-memory or distributed environments. The typically workflow for processing data lazily which is on a disk:
1. "load" data from disk with a defined chunking
2. schedule operations
3. do the computation
 
Lets try this with a simple example:

In [ ]:
import dask.array as da

"Load" the data, by generating a big image with random data

In [ ]:
data = da.random.random(size=(50000, 50000), chunks=(1000, 1000))

Schedule operation, firsty taking the square root, then summing

In [ ]:
data_output = da.sqrt(data).sum()

**Steps 1 and 2 are very fast**, because nothing is actually done, other than initialising and scheduling the tasks to be performed.

Do the actual calculation, using `compute`

In [ ]:
data_output.compute()

**Step 3 is slow**, because all the computation is performed at this stage. Most of the time, this is signficantly slower than in-memory processing, because the chunks of data needs to be read and written from/to disk on request of the scheduler.

This type of processing is very powerful when working with large datasets, but requires some knowledge to use properly.

For more information about dask and its principle see http://matthewrocklin.com/slides/plotcon-2016.html. However, we're jumping onto the next step: how you can use this type of functionality in HyperSpy.

## 2. Loading data lazily

The implementation of out-of-core computation in HyperSpy aims to make processing very large data (not fitting into memory) as seamlessly as possible and similar to in-memory data. This tutorial covers the main difference between lazy and non-lazy processing as well as technicalities you need to be aware of to optimise performance.

As usual, we start by setting up the matplotlib backend and importing hyperspy

In [ ]:
%matplotlib widget
import hyperspy.api as hs

For this tutorial we are going to start by loading 4D-STEM dataset, `lazy_dataset.hspy`. Note that its size is reduced quite a bit, to make it easier to download. The full dataset is `(256 x 256)` probe positions with `(256 x 256)` detector pixels, acquired at ePSIC a couple of years ago. The full dataset can be found at the Zenodo deposit, https://zenodo.org/record/3479124. The file itself: https://zenodo.org/record/3479124/files/fig1_021_sto_12bit_256x256_CL_2_5_1000us.emd?download=1.

In [ ]:
s = hs.load("lazy_dataset.hspy", lazy=True)

Let's check what sort of object we have stored in the ``s`` variable

In [ ]:
print(s)

This is a scanning diffraction dataset with `(144 x 144)` probe positions, and `(144 x 144)` detector pixels.

In [ ]:
# Use the "nbytes" attribute of the numpy array to calculate the size on disk
print(s.data.nbytes / 1e9)

That is about 0.8 GB of data, which we actually could comfortably load into memory and process the standard way. However, we'll use this to show how this type of lazy processing can be done in HyperSpy.

If you want to try this on a much bigger dataset after the workshop, you can check out this [Zenodo deposit](https://zenodo.org/record/4312960), specifically the [largest file](https://zenodo.org/record/4312960/files/fe60al40_stripe_pattern.hspy?download=1), which is a magnetic [STEM-DPC](https://en.wikipedia.org/wiki/Scanning_transmission_electron_microscopy#Differential_phase_contrast) dataset.

## 3. Plotting lazily 

To have a look at the data, we use `s.plot`, just as a non-lazy signal.

In [ ]:
s.plot()

To create the navigation image, just the center part of the diffraction pattern is used. This to reduce the amount of time it takes to generate the navigation image.

Note that there are some big improvements coming in the next version of HyperSpy with regards to plotting lazy signals.

This navigator is stored in `s.navigator`:

In [ ]:
s.navigator.plot()

If we rather want a more annular dark-field (ADF) like contrast, we can utilize the region of interest functionality. Here, we use the `CircleROI` with an inner radius.

In [ ]:
adf_roi = hs.roi.CircleROI(cx=72, cy=72, r=72, r_inner=60)

We can then make a new signal, `s_adf_sum` utilizing the `adf_roi`, the `nansum` function, and `.T`

In [ ]:
s_adf = adf_roi(s, axes=(2, 3))
s_adf_sum = s_adf.nansum(axis=(2, 3), rechunk=False)
s_adf_sum = s_adf_sum.T

Notice that all of these operations are instantaneous, to actually do the calculations, use `.compute()`.

Thanks to the lazy processing, we never have to load the full dataset into memory. So you can potentially do this to datasets which are much larger than your available memory.

In [ ]:
s_adf_sum.compute()

Now we can set it as the navigator for `s`

In [ ]:
s.navigator = s_adf_sum

In [ ]:
s.plot()

## 4. Chunking

An important aspect of lazy processing is **chunking**. This is how the data is organized inside files, like `lazy_dataset.hspy`.

For our 4-dimensional dataset here, the data is split into many smaller 4-dimensional chunks. To see this structure, we use `s.data`

In [ ]:
s.data

The important part is the `Chunk (16, 16, 16, 16)`, which means each chunk consist of `16 x 16` probe positions, and `16 x 16` detector pixels. Each time we want to access something inside a chunk, we need to load the whole chunk into memory.

So for example, if we want to see what the value is for a single detector pixel at one specific probe position, we need to really get the full chunk. For example:

In [ ]:
s_single = s.inav[0, 0].isig[0, 0]
s_single.compute()

Requires just as much reading from the harddrive as reading the full chunk:

In [ ]:
s_single = s.inav[0:32, 0:32].isig[0:32, 0:32]
s_single.compute()

Chunking is quite tricky, with there not being an "ideal" chunking strategy. There are always trade-offs. For now, we'll have a look to why this file is chunked this way.

It makes it very easy to use `transpose` to flip the navigation dimensions, utilizing the same file. This means we can easily navigate the dataset as a function of detector pixels, instead of as a function of probe positions.

In [ ]:
s_t = s.T

In [ ]:
s_t.plot()

We can also navigate a bit quicker using `navigator="slider"`

In [ ]:
s.T.plot(navigator="slider")

## 5. Data reduction through rebinning

One common way of exploring these large datasets, is through reducing their size so that they can fit inside the memory. One easy way of doing this is through `rebin`. By using `scale=(2, 2, 2, 2,)`, we reduce the number of probe positions by 4, and reduce the number of detector pixels by 4.

In [ ]:
s_rebin = s.rebin(scale=(2, 2, 2, 2), rechunk=False)

In [ ]:
s_rebin.data

In [ ]:
print(s_rebin)

In [ ]:
print(s_rebin.data.nbytes / 1e9)

The dataset is now about 200 MB, which is due to reducing the number of data points 16 times, and increasing the bit depth to avoid losing information.

Now we can finally compute it, to load the reduced dataset into memory

In [ ]:
s_rebin.compute()

`s_rebin` is now a non-lazy signal, with its data loaded into memory.

In [ ]:
print(s_rebin)

In [ ]:
s_rebin.plot()

Or look at the transpose

In [ ]:
s_rebin.T.plot()

## 6. Processing the data using `s.map`

To process the data, we can use the `s.map` function, which can apply arbitrary functions to each probe positions.

Lets try to extract some more information from the diffraction patterns, by using edge detection. Here, we can utilize scikit-image's feature functions. For example: `skimage.feature.canny`.

While we could pass the function directly to the `map` function, lets make our own custom function with some pre-processing.

Firstly, import the canny function.

In [ ]:
from skimage.feature import canny

Then we make our function, which we will pass to `map`. 

In [ ]:
def canny_edge_function(image):
    image = canny(image.astype("float32"), sigma=2)
    return image

Then we pass the function to `s.map`. Where `inplace=False` means the function does not alter the original signal `s`. 

In [ ]:
s_canny = s.map(canny_edge_function, inplace=False)

This returns a lazy signal, meaning the calculations haven't really been done yet.

In [ ]:
print(s_canny)

Running `compute()`, we do the actually calculcations.

In [ ]:
s_canny.compute()

Note that you can also save this lazy signal directly, without loading it into memory. This is useful when you're working with really big datasets.

Having run `compute`, we now get a non-lazy signal, which we can plot.

In [ ]:
s_canny.plot()

# 6. Summary

Most operations can be performed *lazily* in HyperSpy:
1. Visualisation
2. Slicing and indexing
3. Generic mathematical operations
4. Machine learning
5. Curve fitting

See [the big data section](https://hyperspy.readthedocs.io/en/stable/user_guide/big_data.html#limitations) of the HyperSpy documentation for more information and to learn about the main difference between lazy and non-lazy signal.